In [2]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [3]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000001E909730EE0>)

In [4]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [5]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [6]:
import json

llm_output_string = "{\"x\": 5, \"y\" : 2}" #output from LLM
llm_output_dict = json.loads(llm_output_string) # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [7]:
print(add.func(**llm_output_dict), "\n",
exponentiate.func(**llm_output_dict))

7 
 25


We're going to construct a simple tool calling agent. We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's _"notes"_ as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant"),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [9]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    model_name = "qwen/qwen3-32b",
    temperature = 0.0
)

When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older `ConversationBufferMemory` class rather than the newer `RunnableWithMessageHistory` — the reason being that we will also be using the older `create_tool_calling_agent` and `AgentExecutor` method and class.

In the `05` chapter we will be using the newer `RunnableWithMessageHistory` class as we'll be building a custom `AgentExecutor`.

In [10]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history", # algins with messagePlaceholder variable_name
    return_messages = True

)

C:\Users\Harivenkat\AppData\Local\Temp\ipykernel_20856\1669027029.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm = llm,
    tools = tools,
    prompt = prompt
)

In [12]:
agent.invoke({
    "input" : "What is 11.676 multiplied by 2.32?",
    "chat_history" : memory.chat_memory.messages,
    "intermediate_steps" : [] # agent will append its internal steps.
    }
)

[ToolAgentAction(tool='multiply', tool_input={'x': 11.676, 'y': 2.32}, log="\nInvoking: `multiply` with `{'x': 11.676, 'y': 2.32}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking to multiply 11.676 by 2.32. Let me check the available functions. There\'s a multiply function that takes two numbers, x and y. So I need to call that. The parameters are x and y, both numbers. The user provided 11.676 and 2.32. I should structure the tool call with these values. Let me make sure I have the function name right. The description says "Multiply \'x\' and \'y\'", so the name is "multiply". The arguments should be x: 11.676 and y: 2.32. I need to present this in JSON format within the tool_call tags. Let me double-check the syntax to avoid any errors.\n', 'tool_calls': [{'id': 'pfe8x9hwb', 'function': {'arguments': '{"x":11.676,"y":2.32}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 

Here, we can see the LLM has generated that we should use the `multiply` tool and the tool input should be `{"x": 10.7, "y": 7.68}`. However, the tool is not executed. For that to happen we need an agent execution loop, which will handle the multiple iterations of generation to tool calling to generation, etc.

We use the `AgentExecutor` class to handle the execution loop:

In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = True
)

In [14]:
agent_executor.invoke({
    "input" : "What is 11.676 multiplied by 2.32?",
    "chat_history" : memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 11.676, 'y': 2.32}`


27.08832The result of multiplying 11.676 by 2.32 is **27.08832**.

> Finished chain.


{'input': 'What is 11.676 multiplied by 2.32?',
 'chat_history': [HumanMessage(content='What is 11.676 multiplied by 2.32?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 11.676 by 2.32 is **27.08832**.', additional_kwargs={}, response_metadata={})],
 'output': 'The result of multiplying 11.676 by 2.32 is **27.08832**.'}

In [15]:
11.676 * 2.32

27.08832

In [16]:
agent_executor.invoke({
    "input" : "My name is Hari",
    "chat_history" : memory
})



> Entering new AgentExecutor chain...
Hello, Hari! How can I assist you today?

> Finished chain.


{'input': 'My name is Hari',
 'chat_history': [HumanMessage(content='What is 11.676 multiplied by 2.32?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 11.676 by 2.32 is **27.08832**.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Hari', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello, Hari! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Hello, Hari! How can I assist you today?'}

In [17]:
agent_executor.invoke({
    "input" : "What is 9+10-(4*2)**3",
    "chat_history" : memory
})



> Entering new AgentExecutor chain...
The result of the expression $9 + 10 - (4 \times 2)^3$ is **-493**. 

**Step-by-step breakdown:**
1. **Parentheses:** $4 \times 2 = 8$
2. **Exponent:** $8^3 = 512$
3. **Addition/Subtraction:** $9 + 10 - 512 = 19 - 512 = -493$

> Finished chain.


{'input': 'What is 9+10-(4*2)**3',
 'chat_history': [HumanMessage(content='What is 11.676 multiplied by 2.32?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 11.676 by 2.32 is **27.08832**.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Hari', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello, Hari! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is 9+10-(4*2)**3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of the expression $9 + 10 - (4 \\times 2)^3$ is **-493**. \n\n**Step-by-step breakdown:**\n1. **Parentheses:** $4 \\times 2 = 8$\n2. **Exponent:** $8^3 = 512$\n3. **Addition/Subtraction:** $9 + 10 - 512 = 19 - 512 = -493$', additional_kwargs={}, response_metadata={})],
 'output': 'The result of the expression $9 + 10 - (4 \\times 2)^3$ is **-493**. \n\n**Step-by-step breakdown:**\n1. **Parenth

In [18]:
9+10-(4*2)**3

-493

In [19]:
agent_executor.invoke({
    "input" : "What is my name",
    "chat_history" : memory
})



> Entering new AgentExecutor chain...
Your name is **Hari**! 😊 How can I assist you today, Hari?

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='What is 11.676 multiplied by 2.32?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of multiplying 11.676 by 2.32 is **27.08832**.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Hari', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello, Hari! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is 9+10-(4*2)**3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of the expression $9 + 10 - (4 \\times 2)^3$ is **-493**. \n\n**Step-by-step breakdown:**\n1. **Parentheses:** $4 \\times 2 = 8$\n2. **Exponent:** $8^3 = 512$\n3. **Addition/Subtraction:** $9 + 10 - 512 = 19 - 512 = -493$', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is **Hari**! 😊 

In [20]:
from langchain.agents import load_tools
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

toolbox = load_tools(tool_names = ['serpapi'], llm = llm)

In [25]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geo location based on the IP"""

    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined"

    except Exception as e:
        return f"Error : {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time"""

    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{input}"),
    ("placeholder" , "{agent_scratchpad}")
])

In [27]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

agent = create_tool_calling_agent(
    llm = llm,
    tools = tools,
    prompt = prompt
)

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)

In [28]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`
responded: 


2025-08-13 18:41:24
Invoking: `get_location_from_ip` with `{}`
responded: 


Latitude: 11.0055,
Longitude: 76.9661,
City: Coimbatore,
Country: IN
Invoking: `Search` with `current weather Coimbatore, IN degrees Celsius`


{'type': 'weather_result', 'temperature': '26', 'unit': 'Celsius', 'precipitation': '0%', 'humidity': '78%', 'wind': '16 km/h', 'location': 'Coimbatore, Tamil Nadu, India', 'date': 'Wednesday 6:00 PM', 'weather': 'Cloudy'}The current date and time is **August 13, 2025, 6:41:24 PM**.  

The weather in **Coimbatore, Tamil Nadu, India** (as of 6:00 PM) is:  
- **Temperature**: 26°C  
- **Precipitation**: 0%  
- **Humidity**: 78%  
- **Wind**: 16 km/h  
- **Conditions**: Cloudy  

Let me know if you need further details! ☁️

> Finished chain.


In [29]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

The current date and time is **August 13, 2025, 6:41:24 PM**.  

The weather in **Coimbatore, Tamil Nadu, India** (as of 6:00 PM) is:  
- **Temperature**: 26°C  
- **Precipitation**: 0%  
- **Humidity**: 78%  
- **Wind**: 16 km/h  
- **Conditions**: Cloudy  

Let me know if you need further details! ☁️